Rather than the DDPG algorithm, try the REINFORCE algorithm

HELPFUL LINKS
My REINFORCE hw: https://github.com/jendawkins/ecehw3/blob/master/Assignment%203%20-%20Policy%20Gradient%20(DDPG%20and%20REINFORCE)%20(2).ipynb

Julia Cartpole REINFORCE: https://github.com/CarloLucibello/DeepRLexamples.jl/blob/master/examples/actor_critic_pong.jl

Brian: do you want to help with debugging this and I'll move on to the next step?

In [1]:
using Pkg

In [2]:
Pkg.add("Gym")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %]  67.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [3]:
Pkg.add("Knet")

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [4]:
Pkg.add("Distributions")

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [5]:
Pkg.add("Flux")

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [6]:
using Flux
using Gym
using Distributions

┌ Info: Recompiling stale cache file /Users/jenniferdawkins/.julia/compiled/v1.0/Flux/QdkVy.ji for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1184


In [7]:
using Knet

In [8]:
using Statistics

In [9]:
env = GymEnv("BipedalWalker-v2");

In [10]:
s_n = env.observation_space.shape[1]; # continuous state space
a_n = env.action_space.shape[1]; # continuous action space
ha_n = [25,25];


Model mlp takes in state and outputs normal distribution over states and actions

In [122]:
x = [-1,-2,-3]
maximum(push!(x,0))
pop!(x)
x

3-element Array{Int64,1}:
 -1
 -2
 -3

In [11]:
struct History
    states
    actions
    rewards
    log_odds
end

In [33]:
h = History(zeros(Float32,0),zeros(Float32,0),zeros(Float32,0),zeros(Float32,0))

History(Float32[], Float32[], Float32[], Float32[])

In [4]:
h.actions

0-element Array{Float32,1}

In [13]:
function relu(x::Any)
    return maximum(push!([x],0.0))
#     pop!(x)
end

relu (generic function with 1 method)

In [14]:
function softplus(x)
    return log(1+exp(x))
end

softplus (generic function with 1 method)

In [30]:
relu(3.0)

3.0


In [15]:
function initweights(hidden, xsize=4, ysize=2)
    w = []
    x = xsize
    for y in [hidden...]
        push!(w, xavier(y, x))
        push!(w, zeros(y, 1))
        x = y
    end
    push!(w, xavier(ysize, x)) # Prob actions
    push!(w, zeros(ysize, 1))
    
    return w
end

initweights (generic function with 3 methods)

In [16]:
function predict(w, x)
    for i = 1:2:length(w)-2
        x = relu.(w[i] * x .+ w[i+1])
    end
    probs = w[end-1] * x .+ w[end]
    return probs
end

predict (generic function with 1 method)

In [14]:
mlp = Chain(
    Dense(s_n, ha_n[1], relu),
    Dense(ha_n[1], ha_n[2], relu),
    BatchNorm(ha_n[2]),
    Dense(ha_n[2], 2))

Chain(Dense(24, 25, relu), Dense(25, 25, relu), BatchNorm(25), Dense(25, 2))

In [17]:
function sample_action(logit,a_n)
    dist = Normal(logit[1],softplus(logit[2]));
    action = rand(dist,a_n);
    log_odds = log.(pdf(dist,action));
    return action, log_odds
end

sample_action (generic function with 1 method)

In [53]:
function update_policy(w, h, gamma=1.0)
    println(typeof(h))
    num_paths = length(h.rewards);
    
    rew_cums = [];
    lo = [0];
    policy_loss = [];
    p, V = predict(w, h.states)
    for p_idx in num_paths
        R = 0;
        for r in reverse(history.reward[p_idx])
            R = r + gamma*R;
            pushfirst!(rew_cums, R)
        end
        append!(lo,history.log_odds[p_idx])
    end
    rew_cums = (rew_cums .- mean(rew_cums)) ./ (std(rew_cums) + 1e-5);
    policy_loss = -mean(lo.*rew_cums);
    return policy_loss
end

update_policy (generic function with 2 methods)

Run Reinforce

In [54]:
n_iter = 1000;
avg_reward = 0;
avg_rewards = [];
step_list_rinforce = [];
total_steps = 0;
episodes = 0;
lr = 1e-3
min_timesteps_per_batch = 2000;
w = initweights(ha_n,s_n,2);
opt = [Adam(lr = lr) for _=1:length(w)];

for itr in 1:n_iter
    paths = [];
    steps = 0;
    batch_rew = 0;
    while steps < min_timesteps_per_batch
        ob = reset!(env)
        obs, acs, rews, log_odds = [],[],[],[];
        done = false;
        ep_rew = 0;
        while ~done
            policy_dist = predict(w,ob)
            action, log_odd = sample_action(policy_dist, a_n)
            next_state, reward, done, information = step!(env, action)
            append!(obs,ob)
            append!(acs,action)
            append!(rews,reward)
            append!(log_odds,log_odd)
            ob = next_state;
            steps +=1;
            ep_rew += reward
            if done
                episodes += 1;
                break
            end
        
        end
        batch_rew += ep_rew;
        append!(h.states, obs)
        append!(h.actions,acs)
        append!(h.rewards,rews)
        append!(h.log_odds,log_odds)
#         path = Dict("observation"=>obs,
#             "reward"=>rews,
#             "action"=>acs,
#             "log_odds"=>log_odds);
#         append!(paths, path)
    end
    println(batch_rew)
    avg_reward += 0.1* batch_rew + avg_reward*0.9;
#     cumulative_reward, policy_loss = update_policy(paths);
    println(avg_reward)
    dw = grad(update_policy)(h)
    update!(w,dw,opt)
end

-278.76446235711313
-27.876446235711313
Param{History}

Stacktrace:
 [1] getproperty(::Any, ::Symbol) at ./sysimg.jl:18
 [2] update_policy(::Param{History}, ::Float64) at ./In[53]:3
 [3] update_policy(::Param{History}) at ./In[53]:2
 [4] #differentiate#1(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::Function, ::Param{History}) at /Users/jenniferdawkins/.julia/packages/AutoGrad/eAmjh/src/core.jl:54
 [5] differentiate(::Function, ::Param{History}) at /Users/jenniferdawkins/.julia/packages/AutoGrad/eAmjh/src/core.jl:45
 [6] (::getfield(AutoGrad, Symbol("##gradfun#6#7")){typeof(update_policy),Int64,Bool})(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::History) at /Users/jenniferdawkins/.julia/packages/AutoGrad/eAmjh/src/core.jl:139
 [7] (::getfield(AutoGrad, Symbol("#gradfun#8")){getfield(AutoGrad, Symbol("##gradfun#6#7")){typeof(update_policy),Int64,Bool}})(::History) at /Users/jenniferdawkins/.julia/packages/A

ErrorException: type Param has no field rewards

In [55]:
size(h.states)

(795768,)

In [25]:
dist = Normal(0,4)
log(pdf(dist, .3);
action = rand(dist,a_n)
log.(pdf(dist,action))

┌ Warning: `pdf(d::UnivariateDistribution, X::AbstractArray)` is deprecated, use `pdf.(d, X)` instead.
│   caller = top-level scope at In[25]:4
└ @ Core In[25]:4


4-element Array{Float64,1}:
 -2.4038529024248714
 -2.33690898767053  
 -3.513541775242507 
 -3.3506354417065   

In [26]:
rand(3,3).*(2).-1

3×3 Array{Float64,2}:
 -0.050383    0.360395   0.984606
  0.0547567  -0.224643   0.585206
  0.791885    0.373463  -0.857316

In [11]:
episode_count = 2000;
min_epsilon = 0.01;
decay_rate = 5/episode_count;

0.0912641486376598

In [2]:
# General Algorithm for Q learning

# For each step in each episode: 
#     Use policy defined by training network to pick actions
#     Update experience replay
# After each episode: 
# Sample from experience replay
# Update training net; (update target net after x episodes)

# using Gym
reward = 0;
episode_count = 2000;
min_epsilon = 0.01;
decay_rate = 5/episode_count;

for i=1:episode_count
    # set epsilon for exploring policy
    epsilon = min_epsilon + (1.0 - min_epsilon)*exp(-decay_rate*i)
    total = 0
    ob = reset!(env)
#     render(env)#comment out this line if you do not want to visualize the environment
    while true
        Q_values = m(ob)
        # exploration
        if epsilon > Random.rand()
            action = Random.rand(env.action_space.shape[1])
        else
            # take max of Q_values
#         action = sample(env.action_space)
        end
        ob, reward, done, information = step!(env, action)
        total += reward
#         render(env)#comment out this line if you do not want to visualize the environment
        done && break
    end
    # Experience reply
    # update training net 
    i%10 && 
    println("episode $i total Rewards: $total")
end

episode 1 total Rewards: -85.97788289192233
episode 2 total Rewards: -98.68037809058097
episode 3 total Rewards: -92.41382523298962
episode 4 total Rewards: -99.93802021494935
episode 5 total Rewards: -79.69650269939272
episode 6 total Rewards: -102.57114505678385
episode 7 total Rewards: -110.65297324995726
episode 8 total Rewards: -100.0879210012217
episode 9 total Rewards: -98.46423048953291
episode 10 total Rewards: -85.04734643681674


In [13]:
sample(env.action_space)

4-element Array{Float64,1}:
  0.5131067509384377 
  0.8626462959402228 
 -0.3850172680270889 
  0.40907506858505505

In [30]:
env.action_space.shape[1]

4

In [31]:
Random.rand(env.action_space.shape[1])

4-element Array{Float64,1}:
 0.5091860357094704 
 0.46913460209260993
 0.03589676414598042
 0.27638164170088775

In [ ]:
# State consists of hull angle speed, angular velocity, horizontal speed, vertical speed,
# position of joints and joints angular speed, legs contact with ground, and 10 lidar
# rangefinder measurements to help to deal with the hardcore version. There's no coordinates
# in the state vector. Lidar is less useful in normal version, but it works.

In [5]:
env.observation_space

BoxS(Float32[-Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf  …  -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf], Float32[Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf  …  Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf], (24,))

In [3]:
env.action_space
# I think this is force on 4 joints
# limits: -1 to 1

BoxS(Float32[-1.0, -1.0, -1.0, -1.0], Float32[1.0, 1.0, 1.0, 1.0], (4,))

In [44]:
env.action_space.sample()

ErrorException: type BoxS has no field sample